# 📦 1. Importar librerías necesarias


In [ ]:
import requests
from time import sleep
import json
from datetime import timedelta
import pandas as pd

# 🔐 2. Configurar clave API y headers

In [ ]:
# API Key 
api_key = 'REMOVED_API_KEY'

# Creamos los headers que exige PUBG para autenticar y especificar el formato
headers = {
    "Authorization": f"Bearer {api_key}",
    "Accept": "application/vnd.api+json"
}

# 🎮 3. Buscar el ID del jugador

In [ ]:
# URL base de la API de PUBG 
base_url = "https://api.pubg.com/shards/steam"

# Nombre del jugador a buscar
player_name  = 'arbigegegegegege'
#player_name  = 'KaDiz-'
#player_name  = 'XioYuanLinPay'

# Endpoint para buscar jugadores por nombre
url = f"{base_url}/players?filter[playerNames]={player_name}"

# Hacemos la petición
response = requests.get(url, headers=headers)

# Mostramos el código de respuesta para verificar que todo va bien (200 = OK)
print("Código de estado:", response.status_code)

# 🎮 3.5 Respuesta y usuario encontrado

In [ ]:
# Si todo va bien, extraemos el ID del jugador
if response.status_code == 200:
    data = response.json()
    dataOriginal = response.json()
    player_id = data["data"][0]["id"]
    print(f"ID del jugador {player_name}: {player_id}")
else:
    print("Error al obtener los datos:", response.text)

# Pequeña prueba para ver como es la estructura del json.

In [ ]:
print(json.dumps(data, indent=4))

# 🎯 4. Obtener matches jugados por el jugador

In [ ]:
# Endpoint para obtener datos del player por id
url_id = f"https://api.pubg.com/shards/steam/players/{player_id}"

# Hacemos la petición
response_id = requests.get(url_id, headers=headers)

# Convertimos la respuesta a JSON
data_id = response_id.json()

# Accedemos a los matches jugados por el jugador
matches = data_id["data"]["relationships"]["matches"]["data"]
for i, match in enumerate(matches[:10]):
    print(f"Match {i+1}: {match['id']}")

# Guardamos los match_ids en la lista (Solo guardamos 1 de momento para probar)
match_ids = [match["id"] for match in matches[:10]]
last_Match_id = [match["id"] for match in matches[:1]]

# 📊 5. Mostrar detalles del primer match del jugador

In [ ]:
# Recorremos cada match_id para obtener información de la partida
for match_id  in match_ids:
    # Endpoint para obtener datos del match
    url = f"https://api.pubg.com/shards/steam/matches/{match_id}"
    
    # Hacemos la petición para obtener los datos de la partida
    response = requests.get(url, headers=headers)
    
    # Convertimos la respuesta a JSON
    match_data = response.json()
    
    # Accedemos a la información del jugador dentro de esa partida
    participants = match_data["included"]

    # Buscamos al jugador en esa partida (comparando player_id)
    for player in participants:
        if player["type"] == "participant" and "stats" in player["attributes"]:
            stats = player["attributes"]["stats"]
           
            # Verifica si el jugador es el que estás buscando
            if stats.get("playerId") == player_id:
                kills = stats.get("kills", 0)
                name_player = stats.get("name", "")
                killPlace = stats.get("killPlace",0)
                killStreaks = stats.get("killStreaks",0)
                damage = stats.get("damageDealt", 0)                
                duracion = timedelta(seconds=stats.get("timeSurvived", 0))
                ranking = stats.get("winPlace", 0)
                win = ranking == 1
                
                # Imprimir los resultados de la partida
                print(f"Match ID: {match_id}")
                print(f"Player Name: {name_player}")
                print(f"Kills: {kills}")
                print(f"killPlace: {killPlace}")
                print(f"killStreaks: {killStreaks}")
                print(f"Damage: {damage}")
                print(f"Time survived: {duracion}")
                print(f"Ranking: {ranking}")
                print(f"Win: {'Yes' if win else 'No'}\n")

In [ ]:
# Variables para acumular los resultados
total_kills = 0
total_damage = 0
total_time_survived = 0
total_ranking = 0
total_wins = 0
total_kill_place = 0
total_kill_streaks = 0
num_matches = 0  # Para contar el número de partidas procesadas

# Recorremos cada match_id para obtener información de la partida
for match_id  in match_ids:
    # Endpoint para obtener datos del match
    url = f"https://api.pubg.com/shards/steam/matches/{match_id}"
    
    # Hacemos la petición para obtener los datos de la partida
    response = requests.get(url, headers=headers)
    
    # Convertimos la respuesta a JSON
    match_data = response.json()
    
    # Accedemos a la información del jugador dentro de esa partida
    participants = match_data["included"]

    # Buscamos al jugador en esa partida (comparando player_id)
    for player in participants:
        if player["type"] == "participant" and "stats" in player["attributes"]:
            stats = player["attributes"]["stats"]
           
            # Verifica si el jugador es el que estás buscando
            if stats.get("playerId") == player_id:
                kills = stats.get("kills", 0)
                killPlace = stats.get("killPlace", 0)
                killStreaks = stats.get("killStreaks", 0)
                damage = stats.get("damageDealt", 0)
                duracion = timedelta(seconds=stats.get("timeSurvived", 0))
                ranking = stats.get("winPlace", 0)
                win = ranking == 1
                
                # Acumular los valores
                total_kills += kills
                total_damage += damage
                total_time_survived += stats.get("timeSurvived", 0)
                total_ranking += ranking
                total_wins += win
                total_kill_place += killPlace
                total_kill_streaks += killStreaks
                num_matches += 1

# Calcular las medias
if num_matches > 0:
    avg_kills = total_kills / num_matches
    avg_damage = total_damage / num_matches
    avg_time_survived = timedelta(seconds=(total_time_survived / num_matches))
    avg_ranking = total_ranking / num_matches
    avg_wins = total_wins / num_matches
    avg_kill_place = total_kill_place / num_matches
    avg_kill_streaks = total_kill_streaks / num_matches

    # Mostrar las medias
    print("Average Stats over the last 10 matches:")
    print(f"Average Kills: {avg_kills:.2f}")
    print(f"Average Damage: {avg_damage:.2f}")
    print(f"Average Time Survived: {avg_time_survived}")
    print(f"Average Ranking: {avg_ranking:.2f}")
    print(f"Average Wins: {avg_wins:.2f}")
    print(f"Average Kill Place: {avg_kill_place:.2f}")
    print(f"Average Kill Streaks: {avg_kill_streaks:.2f}")
else:
    print("No matches found.")

# 📊 5.5 Mostrar detalles del primer match obteniendo todos los jugadores y equipos

In [ ]:
# Recorremos cada match_id para obtener información de la partida
for match_id  in last_Match_id:
    # Endpoint para obtener datos del match
    url = f"https://api.pubg.com/shards/steam/matches/{match_id}"
    
    # Hacemos la petición para obtener los datos de la partida
    response = requests.get(url, headers=headers)
    
    # Convertimos la respuesta a JSON
    match_data = response.json()
    
    # Buscamos al jugador en esa partida (comparando player_id)
    roster_match = match_data["included"]
    all_participants = []  # ← Aquí guardaremos todos los participantes

    # Primero, recogemos todos los participantes
    for data in roster_match:
        if data["type"] == "participant":
            all_participants.append(data)

    # Usamos un set para llevar control de los jugadores ya procesados y evitar duplicados
    seen_players = set()

    # Suponiendo que tienes una lista de rosters
    for data in roster_match:  
        if data["type"] == "roster":
            team_id = data["id"]
            team_rank = data["attributes"]["stats"].get("rank", "Desconocido")
            team_Id = data["attributes"]["stats"].get("teamId", "Desconocido")
            team_win = data["attributes"]["won"] == "true"
            participants = data["relationships"]["participants"]["data"]

            print(f"Match ID: {match_id}")
            print(f"team id: {team_id}")
            print(f"team nº: {team_Id}")
            print(f"team rank: {team_rank}")
            print(f"Win?: {'Yes' if team_win else 'No'} \n")   

            #Buscar jugadores en el roster y sus estadísticas
            for player_info in participants:
                player_id = player_info["id"]  # ID del jugador en el roster
                
                # Evitamos imprimir jugadores repetidos
                if player_id in seen_players:
                    continue
                seen_players.add(player_id)  # Lo marcamos como "visto"

                # Buscar el participante con ese ID
                player = next((p for p in all_participants if p["id"] == player_id), None)

                if not player or "attributes" not in player or "stats" not in player["attributes"]:
                    continue

                stats = player["attributes"]["stats"]
                kills = stats.get("kills", 0)
                name_player = stats.get("name", "Desconocido")
                damage = stats.get("damageDealt", 0)
                ranking = stats.get("winPlace", 0)
                duracion = timedelta(seconds=stats.get("timeSurvived", 0))
                win = ranking == 1

                # Mostrar detalles del jugador
                print(f"Player ID: {player_id}")
                print(f"Player Name: {name_player}")
                print(f"Kills: {kills}")
                print(f"Damage: {damage}")
                print(f"Time survived: {duracion}")
                print(f"Ranking: {ranking} \n")

# Adaptando el codigo a Pandas

In [ ]:
print(json.dumps(dataOriginal, indent=4))

In [ ]:
# Suponiendo que tu variable es 'data' 
players = dataOriginal["data"]

# Normalizamos todo lo posible, separando las claves anidadas con "_"
df = pd.json_normalize(players, sep="_")

# Mostramos el DataFrame completo
df


In [ ]:
# Normalizar los matches dentro de relationships.matches.data
df_matches = pd.json_normalize(
    players,
    record_path=["relationships", "matches", "data"],
    sep="_"
)

# Mostramos el DataFrame completo
df_matches

In [ ]:
# URL para obtener las seasons
urlSeasonActual = "https://api.pubg.com/shards/steam/seasons"


# Hacemos la petición
responseActualSeason = requests.get(urlSeasonActual, headers=headers)

# Verificamos y extraemos el seasonId actual
if responseActualSeason.status_code == 200:
    seasons = responseActualSeason.json()
    
    # Buscamos la temporada actual
    season_id = None
    for s in seasons["data"]:
        if s["attributes"].get("isCurrentSeason"):
            season_id = s["id"]
            break

    if season_id is None:
        print("No se encontró la temporada actual.")
    else:
        # Armamos el endpoint con el season actual
        endpointSeason = f"https://api.pubg.com/shards/steam/players/{player_id}/seasons/{season_id}/ranked"

        # Hacemos la petición
        response = requests.get(endpointSeason, headers=headers)

        if response.status_code == 200:
            dataSeason = response.json()
            print("Todo Correcto, arcivhos guardado en data")
        else:
            print("Error en ranked:", response.status_code)
else:
    print("Error al obtener seasons:", responseActualSeason.status_code)


In [ ]:
print(json.dumps(dataSeason, indent=4))